# Script

In [332]:
import numpy as np
import pandas as pd
from datetime import timedelta

## ICUpatients5

In [333]:
df_patients = pd.read_csv('./data/input/ICUpatients5.csv')
df_patients.head(3)

,subject_id,hadm_id,stay_id,gender,anchor_age,admittime,intime,outtime,deathtime,intervaloAdmIntUTI,tempoUTI,race,pesoadm,unidpesoadm,altura,unidaltura,imc
0,12088486,29875103,39348271,M,18,2135-02-07 02:57:00 UTC,2135-02-07 03:03:00 UTC,2135-02-12 00:25:00 UTC,2135-02-11 00:01:00 UTC,0,117,UNABLE TO OBTAIN,71.8,kg,178.0,cm,-99.9
1,19850244,27972658,32398411,F,18,2162-06-04 17:38:00 UTC,2162-06-05 15:25:42 UTC,2162-06-09 20:22:03 UTC,NaN,21,100,OTHER,57.7,kg,163.0,cm,-99.9
2,10562205,20149612,34317198,M,18,2149-04-29 23:19:00 UTC,2149-04-30 00:58:00 UTC,2149-05-09 20:54:46 UTC,NaN,1,235,WHITE,98.0,kg,180.0,cm,-99.9


In [334]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7493 entries, 0 to 7492
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   subject_id          7493 non-null   int64  
 1   hadm_id             7493 non-null   int64  
 2   stay_id             7493 non-null   int64  
 3   gender              7493 non-null   object 
 4   anchor_age          7493 non-null   int64  
 5   admittime           7493 non-null   object 
 6   intime              7493 non-null   object 
 7   outtime             7493 non-null   object 
 8   deathtime           1379 non-null   object 
 9   intervaloAdmIntUTI  7493 non-null   int64  
 10  tempoUTI            7493 non-null   int64  
 11  race                7493 non-null   object 
 12  pesoadm             7493 non-null   float64
 13  unidpesoadm         7493 non-null   object 
 14  altura              7493 non-null   float64
 15  unidaltura          7493 non-null   object 
 16  imc   

In [335]:
df_patients.describe().T

,count,mean,std,min,25%,50%,75%,max
subject_id,7493.0,1.499547e+07,2.885518e+06,10001884.0,12528884.0,15017391.0,17501651.0,19999840.0
hadm_id,7493.0,2.499762e+07,2.883007e+06,20001361.0,22538287.0,24968948.0,27494646.0,29999625.0
stay_id,7493.0,3.502181e+07,2.896595e+06,30004530.0,32506767.0,35029720.0,37528363.0,39999230.0
anchor_age,7493.0,6.426171e+01,1.661682e+01,18.0,54.0,66.0,77.0,91.0
intervaloAdmIntUTI,7493.0,2.860910e+01,9.290403e+01,-6.0,0.0,1.0,18.0,3825.0
tempoUTI,7493.0,1.965248e+02,1.687363e+02,72.0,95.0,137.0,231.0,2391.0
pesoadm,7493.0,8.365305e+01,2.500577e+01,31.1,67.4,80.0,95.8,345.0
altura,7493.0,1.692113e+02,1.259287e+01,51.0,163.0,170.0,178.0,445.0
imc,7493.0,-9.990000e+01,0.000000e+00,-99.9,-99.9,-99.9,-99.9,-99.9


In [336]:
df_patients.describe(exclude=np.number).T

,count,unique,top,freq
gender,7493,2,M,4368
admittime,7493,7488,2183-10-04 07:15:00 UTC,2
intime,7493,7493,2135-02-07 03:03:00 UTC,1
outtime,7493,7493,2135-02-12 00:25:00 UTC,1
deathtime,1379,1379,2135-02-11 00:01:00 UTC,1
race,7493,33,WHITE,4629
unidpesoadm,7493,1,kg,7493
unidaltura,7493,1,cm,7493


In [337]:
data_types = {
    'subject_id': 'int64',
    'hadm_id': 'int64',
    'stay_id': 'int64',
    'intime': 'datetime',
    'outtime': 'datetime',
}

In [338]:
correct_format = '%Y-%m-%d %H:%M:%S UTC'

for col, dtype in data_types.items():
    if dtype == 'datetime':
        df_patients[col] = pd.to_datetime(df_patients[col], format=correct_format)
    else:
        df_patients[col] = df_patients[col].astype(dtype)

In [339]:
df_patients = df_patients[data_types.keys()]

In [340]:
max_days = np.abs(df_patients.intime - df_patients.outtime).dt.days.max() + 1

In [341]:
df_patients.head(1)

,subject_id,hadm_id,stay_id,intime,outtime
0,12088486,29875103,39348271,2135-02-07 03:03:00,2135-02-12 00:25:00


In [342]:
final_list = []

for i, row in df_patients.iterrows():
    for j in range(max_days):
        start_time = row['intime'] + timedelta(days=j)
        end_time = row['intime'] + timedelta(days=j+1)
        maybe_time = end_time + timedelta(hours=6)
        
        if start_time >= row['outtime']:
            break
        
        if end_time <= row['outtime'] < maybe_time:
            final_list.append([row['subject_id'], j+1, start_time, row['outtime']])
            break
        
        if end_time >= row['outtime']:
            final_list.append([row['subject_id'], j+1, start_time, row['outtime']])
            break
            
        diff = (row['outtime'] - end_time).total_seconds() // 3600
        
        if 6 < diff < 24:
            final_list.append([row['subject_id'], j+1, start_time, end_time])
            final_list.append([row['subject_id'], j+2, end_time, row['outtime']])
            break
            
        final_list.append([row['subject_id'], j+1, start_time, end_time])
            
df_final = pd.DataFrame(final_list, columns=['subject_id', 'day', 'start_time', 'end_time'])

In [343]:
df_final['diff_end_start'] = (df_final.end_time - df_final.start_time).dt.total_seconds() // 3600

In [344]:
df_final.to_csv('./data/output/ICUp5NewWindow24.csv', index=False)